# Data Processing

This tutorial explains how tabular data can be handled and transformed with the `Table` class.

**Important note:** All operations on a `Table` return a new `Table`. The original `Table` will not be changed.

1. Load your data into a `Table`:

In [ ]:
from safeds.data.tabular.containers import Table

titanic = Table.from_csv("data/titanic.csv")

2. Create a `Table` containing only the first 10 rows:

In [ ]:
titanic_slice = titanic.slice(end=10)

titanic_slice # just to show the output

3. Extract a `Row` from your `Table`:

In [ ]:
titanic_slice.get_row(0)

4. Extract a `Column` from your `Table`:

In [ ]:
titanic_slice.get_column("name")

5. Combine a list of `Row`s to a `Table` (make sure the `Row`s have the same columns):

In [ ]:
Table.from_rows([
    titanic_slice.get_row(0),
    titanic_slice.get_row(1)
])

6. Combine a list of `Column`s to a `Table` (make sure the `Column`s have the same amount of rows):

In [ ]:
Table.from_columns([
    titanic_slice.get_column("name"),
    titanic_slice.get_column("age")
])

7. Drop columns from a `Table`:

In [ ]:
titanic_slice.drop_columns([
    "id",
    "name",
    "ticket",
    "cabin",
    "port_embarked",
    "survived"
])

8. Keep only specified columns of a `Table`:

In [ ]:
titanic_slice.keep_columns(["name", "survived"])

9. Sort _columns_ by their name:

In [ ]:
titanic_slice.sort_columns()

10. Sort _columns_ with a custom comparator:

In [ ]:
titanic_slice.sort_columns(
    lambda column1, column2:
        (column1.name < column2.name) - (column1.name > column2.name)
)

11. Filter rows with a given query:

In [ ]:
titanic.filter_rows(
    lambda row:
        "van" in row.get_value("name")
)
